## PySpark 설치

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 655.2 kB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845493 sha256=c1c156e73ff3fce11f70284ca289c77c72c26fd0ef4029d0b2f00c9ceb291557
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
!cd /usr/local/lib/python3.10/dist-packages/pyspark/jars && wget https://redshift-downloads.s3.amazonaws.com/drivers/jdbc/2.1.0.24/redshift-jdbc42-2.1.0.24.jar

--2024-06-19 05:31:16--  https://redshift-downloads.s3.amazonaws.com/drivers/jdbc/2.1.0.24/redshift-jdbc42-2.1.0.24.jar
Resolving redshift-downloads.s3.amazonaws.com (redshift-downloads.s3.amazonaws.com)... 3.5.25.132, 54.231.198.241, 52.216.170.3, ...
Connecting to redshift-downloads.s3.amazonaws.com (redshift-downloads.s3.amazonaws.com)|3.5.25.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1072235 (1.0M) [binary/octet-stream]
Saving to: ‘redshift-jdbc42-2.1.0.24.jar’

redshift-jdbc42-2.1 100%[===================>]   1.02M  --.-KB/s    in 0.1s    

2024-06-19 05:31:16 (6.86 MB/s) - ‘redshift-jdbc42-2.1.0.24.jar’ saved [1072235/1072235]



In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL #1") \
    .getOrCreate()

## Redshift 상의 다음 테이블을 이용하여 월별 채널별 매출과 방문자 정보 계산하기
user_session_channel, session_timestamp, session_transaction

In [5]:
# Redshift와 연결해서 DataFrame으로 로딩하기
url = "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:****/dev?user=guest&password=****"

df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

df_session_transaction = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_transaction") \
    .load()

In [6]:
df_user_session_channel.createOrReplaceTempView("user_session_channel")
df_session_timestamp.createOrReplaceTempView("session_timestamp")
df_session_transaction.createOrReplaceTempView("session_transaction")

In [7]:
df_user_session_channel.show(5)

+------+--------------------+-------+
|userid|           sessionid|channel|
+------+--------------------+-------+
|  1491|00029153d12ae1c9a...|Organic|
|    59|0002ac0d783338cfe...|  Naver|
|   117|0006246bee639c7a7...|Youtube|
|   572|0006dd05ea1e999dd...|Organic|
|   935|0007cda84fafdcf42...| Google|
+------+--------------------+-------+
only showing top 5 rows



In [8]:
df_session_timestamp.show(5)

+--------------------+--------------------+
|           sessionid|                  ts|
+--------------------+--------------------+
|00029153d12ae1c9a...|2019-10-18 14:14:...|
|0004289ee1c7b8b08...|2019-11-16 21:20:...|
|0006246bee639c7a7...|2019-08-10 16:33:...|
|0006dd05ea1e999dd...|2019-07-06 19:54:...|
|000958fdaefe0dd06...|2019-11-02 14:52:...|
+--------------------+--------------------+
only showing top 5 rows



In [9]:
df_session_transaction.show(5)

+--------------------+--------+------+
|           sessionid|refunded|amount|
+--------------------+--------+------+
|00029153d12ae1c9a...|   false|    85|
|008909bd27b680698...|   false|    13|
|0107acb41ef20db22...|   false|    16|
|018544a2c48077d2c...|   false|    39|
|020c38173caff0203...|   false|    61|
+--------------------+--------+------+
only showing top 5 rows



## 총 매출이 가장 많은 사용자 10명 찾기

In [10]:
top_rev_user_df = spark.sql("""
    SELECT userid,
        SUM(str.amount) revenue,
        SUM(CASE WHEN str.refunded = False THEN str.amount END) net_revenue
    FROM user_session_channel usc
    JOIN session_transaction str ON usc.sessionid = str.sessionid
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10""")

In [11]:
top_rev_user_df.show()

+------+-------+-----------+
|userid|revenue|net_revenue|
+------+-------+-----------+
|   989|    743|        743|
|   772|    556|        556|
|  1615|    506|        506|
|   654|    488|        488|
|  1651|    463|        463|
|   973|    438|        438|
|   262|    422|        422|
|  1099|    421|        343|
|  2682|    414|        414|
|   891|    412|        412|
+------+-------+-----------+



In [12]:
top_rev_user_df2 = spark.sql("""
SELECT
  userid,
  SUM(amount) total_amount,
 	RANK() OVER (ORDER BY SUM(amount) DESC) rank
FROM session_transaction st
JOIN user_session_channel usc ON st.sessionid = usc.sessionid
GROUP	BY userid
ORDER BY rank
LIMIT 10""")

In [13]:
top_rev_user_df2.show()

+------+------------+----+
|userid|total_amount|rank|
+------+------------+----+
|   989|         743|   1|
|   772|         556|   2|
|  1615|         506|   3|
|   654|         488|   4|
|  1651|         463|   5|
|   973|         438|   6|
|   262|         422|   7|
|  1099|         421|   8|
|  2682|         414|   9|
|   891|         412|  10|
+------+------------+----+

